In [1]:
import numpy as np

## Reading parameters.out

In [61]:
class ParametersOut:
    def __init__(self, filename):
        self.level_blocks = []
        with open(filename, 'r') as f:
            file = f.read()
            lines = np.array(file.split('\n'))
            blank_line_indices = np.where('' == np.array(lines))[0]
            n = len(blank_line_indices)
            for i in range(n-2):
                block = lines[blank_line_indices[i]+1:blank_line_indices[i+1]]
                self.level_blocks.append(LevelBlock(block))
    
    
class LevelBlock:
    def __init__(self, lines):
        num_lines = len(lines)
        
        first_line = lines[0].split()
        Jpi = first_line[2]
        self.J  = float(Jpi[:-1])
        self.pi = 1 if Jpi[-1] == '+' else -1
        self.E_level = float(first_line[5])
        self.E_level_units = first_line[6]
        
        self.lines = []
        for i in range(1, num_lines):
            self.lines.append(Line(lines[i]))
            
                
                
class Line:
    def __init__(self, line):
        self.values = {}
        self.line = np.array(line.split())
        ii = np.where('=' == self.line)[0] - 1
        n = len(ii)
        for j in range(n-1):
            self.values[self.line[ii[j]]] = ' '.join(self.line[ii[j]+2:ii[j+1]])
        self.values[self.line[ii[n-1]]] = ' '.join(self.line[ii[n-1]+2:])

In [62]:
po = ParametersOut('output/parameters.out')

In [63]:
# for lb in po.level_blocks:
#     for l in lb.lines:
#         for v in l.values:
#             print(v, l.values[v])

## Reading param.errors

In [82]:
with open('output/param.errors', 'r') as f:
    file = f.read()

In [92]:
for l in list(filter(lambda l: l is not '', file.split('\n'))):
    print(l.split())

['j=1_la=1_energy', '4.2910000e-01', '0.0000000e+00', '0.0000000e+00']
['j=1_la=1_ch=1_rwa', '2.9065972e-01', '7.7725719e-01', '1.8436419e-01']
['j=1_la=1_ch=2_rwa', '0.0000000e+00', '0.0000000e+00', '0.0000000e+00']
['j=1_la=1_ch=3_rwa', '-0.0000000e+00', '0.0000000e+00', '0.0000000e+00']
['j=1_la=2_energy', '2.1600000e+01', '0.0000000e+00', '0.0000000e+00']
['j=1_la=2_ch=1_rwa', '-1.0084113e+00', '3.8121127e-01', '3.3325821e-01']
['j=1_la=2_ch=2_rwa', '0.0000000e+00', '0.0000000e+00', '0.0000000e+00']
['j=1_la=2_ch=3_rwa', '0.0000000e+00', '0.0000000e+00', '0.0000000e+00']
['j=2_la=1_energy', '1.4000000e+01', '0.0000000e+00', '0.0000000e+00']
['j=2_la=1_ch=1_rwa', '1.5922551e+00', '1.0552532e-02', '1.0769077e-02']
['j=2_la=1_ch=2_rwa', '1.8390635e+00', '3.8989182e-01', '3.3372383e-01']
['j=2_la=1_ch=3_rwa', '1.3013933e-01', '1.7640600e+00', '1.6894245e+00']
['j=3_la=1_energy', '0.0000000e+00', '0.0000000e+00', '0.0000000e+00']
['j=3_la=1_ch=1_rwa', '1.1838254e+00', '8.1599879e-02', '

In [109]:
class ParamErrors:
    def __init__(self, filename):
        self.params = []
        with open(filename, 'r') as f:
            self.lines = list(filter(lambda l: l is not '', f.read().split('\n')))
            for l in self.lines:
                self.params.append(Param(l))

class Param:
    def __init__(self, line):
        self.line  = line.split()
        self.label = ParamLabel(self.line[0])
        self.value = float(self.line[1])
        self.plus  = float(self.line[2])
        self.minus = float(self.line[3])
        self.varied = (self.plus != 0 or self.minus != 0)
        
        
class ParamLabel:
    def __init__(self, label):
        self.components = label.split('_')
        self.type = self.components[-1]
        s = self.components[0]
        if self.type != 'norm':
            self.j = int(s[2:])

In [110]:
pe = ParamErrors('output/param.errors')